#  Project

**Project Goal:** 

**Dataset Period:**

**Methodology:** 

---

## 1. Business Understanding

### 1.1 Business Objectives
TODO

### 1.2 Project Goals and Successs Criteria
TODO


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler, PowerTransformer
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import (
    BaggingRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)

import math
from statsmodels.nonparametric.smoothers_lowess import lowess
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

---
## 2. Data Understanding

### 2.1 Data Collection

The first step is to load the datasets into the working environment. This involves importing the necessary libraries and reading the data file into a suitable data structure, namely, a DataFrame using Pandas.

In [ ]:
# Load datasets
weather = pd.read_csv("weather.csv", parse_dates=["time"])
energy = pd.read_csv("energy_dataset.csv", parse_dates=["time"])

# Set 'time' as index
weather = weather.set_index("time")
energy = energy.set_index("time")

### 2.2 Data Description

##### Basic dataset information

In [ ]:
energy.info()

The *energy* dataset contains 35064 entries and 24 features, representing hourly records of electricity generation from various sources, total system load, and the day-ahead market price. Each row corresponds to one hour of energy system operation, and the goal is to forecast the electricity price one hour and one day ahead.

In [ ]:
energy.describe()

In [ ]:
energy.head()

In [ ]:
weather.info()

The *weather* dataset also contains 35064 entries and 5 features representing hourly meteorological measurements such as temperature, pressure, humidity, and wind speed. Each row corresponds to one hour of weather conditions, and these variables are used as exogenous inputs to improve electricity price forecasting.

In [ ]:
weather.describe()

In [ ]:
weather.head()

### 2.3 Data Exploration

#### 2.3.1 Target variable analysis

The target variable `price_day_ahead` represents the eletricity market price for the upcoming hour.

In [ ]:
target_col='price_day_ahead'

plt.figure(figsize=(8,5))
plt.hist(energy[target_col], bins=50, edgecolor='black')
plt.title("Histogram of Day-Ahead Electricity Price")
plt.xlabel("Price (€/MWh)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

The histogram shows a distribution that is very close to a normal distribution, although it is slightly right-skewed.

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(energy[target_col])
plt.title("Time Series of Electricity Price (Day-Ahead)")
plt.xlabel("Time")
plt.ylabel("Price (€/MWh)")
plt.grid(True)
plt.show()

The time series plot shows strong short-term fluctuations and clear seasonal patterns, with occasional price spikes. Prices vary over time, indicating non-stationarity and the presence of both volatility and periodic behavior.

#### 2.3.2 Feature distribution analysis

Now we will perform feature distribution analysis to examine how the data values are spread across the datasets. We will use plots and histograms to visualise the distributions features. Box plots for will be skipped in this step, as they will be specifically used for outlier detection in paragraph `2.4.2`.

In [ ]:
def feat_distribution(nonDiscreteFeatures, df):
    nrows = math.ceil(len(nonDiscreteFeatures) / 2)
    ncols = 2
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, nrows * 5))
    axes = axes.flatten()

    for i, col in enumerate(nonDiscreteFeatures):
        df[col].hist(bins=30, ax=axes[i])
        axes[i].set_title(f'{col} Distribution')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')

    plt.show()

All of the features are numerical and continuous.

##### Energy dataset

In [ ]:
energy_features = [col for col in energy.columns 
                   if col not in ['time', 'price_day_ahead']]
feat_distribution(energy_features, energy)

None of the features has a normal distribution. `generation_hydro_pumped_storage_consumption`, `generation_hydro_water_reservoir`, and `generation_solar` are strongly right-skewed, while `generation_wind_onshore` is moderately right-skewed. In contrast, `generation_waste` is left-skewed.
Variables with skewed distributions will be handled in paragraph 3.4.3, because skewness can negatively affect the models used later in paragraph 4.2, particularly KNN, SVR, and Linear Regression, which rely on distance metrics or assumptions of linearity.

`generation_marine`, `generation_geothermal`, `generation_fossil_oil_shale`, `generation_fossil_peat`, and `generation_fossil_coal-derived_gas` contain only zeros, which is consistent with the dataset description provided in paragraph 2.2.

The data represented by the remaining variables generally show irregular, multimodal, or heavily skewed shapes, reflecting the diverse and highly variable nature of electricity generation across different sources.

##### Weather dataset

In [ ]:
weather_features = [col for col in weather.columns if col != 'time']
feat_distribution(weather_features, weather)

`temperature` and `pressure` distribution is close to normal, whereas `humidity` is slightly left-skewed and `wind_speed` is right-skewed.

Skeweness will be handled in paragraph DOK

In [ ]:
skewed_cols = ['generation_hydro_pumped_storage_consumption','generation_hydro_water_reservoir','generation_solar','generation_wind_onshore','generation_waste','wind_speed']

### 2.4 Data Quality Assessment

#### 2.4.1 Identify missing values

In the first place, we will check for missing values to ensure data completeness and avoid potential issues during analysis and modeling.

In [ ]:
# % missing values by column

def missing_values_info(df):
    nulls = df.isnull().sum()
    percent = round(nulls / df.shape[0] * 100, 3)
    
    nullvalues = pd.concat([nulls, percent], axis=1)
    nullvalues.columns = ["Count", "%"]
    
    return nullvalues


In [ ]:
missing_values_info(energy)

The proportion of missing values varies across the energy-generation features. Most variables contain only a very small fraction of missing entries (around 0.05% each), including `generation_biomass`, `generation_fossil_gas`, `generation_solar`, `generation_wind_onshore`, and many others.
A few features have slightly higher but still low missing rates, such as `generation_nuclear` at 0.048% and `total_load_actual` at 0.103%.

Two variables — `generation_hydro_pumped_storage_aggregated` and `forecast_wind_offshore_eday_ahead` — have 100% missing values, meaning they contain no usable data.

In [ ]:
# skipping the NaNs
energy_features = [
    f for f in energy_features
    if energy[f].notna().sum() > 0
]

In [ ]:
missing_values_info(weather)

There are no missing values in `weather` dataset.

#### 2.4.2 Identify outliers

Based on dataframes information in paragraph `2.2` there might be potential outliers such as values at the extreme ends of the distributions (e.g., very high generation levels or unusually low/high prices). These points can disproportionately influence analysis and model results. That's why now we will identify outliers.

DOK 
Rolling Window Analysis: calculate rolling mean and standard deviation. Values
that deviate significantly from the rolling mean (e.g., beyond ±3 standard
deviations) may be considered outliers


##### 2.4.2.1 Target variable

In [ ]:
plt.figure(figsize=(6,5))
plt.boxplot(
    energy[target_col],
    vert=True,
    patch_artist=True,
    boxprops=dict(facecolor='lightblue', color='blue'),
    medianprops=dict(color='red'),
    whiskerprops=dict(color='blue'),
    capprops=dict(color='blue'),
    flierprops=dict(marker='o', markersize=4, markerfacecolor='blue')
)
plt.title("Boxplot of Day-Ahead Electricity Price")
plt.ylabel("Price (€/MWh)")
plt.grid(True)
plt.show()


The boxplot shows clear price spikes, confirming the presence of outliers in the target variable. These extreme values reflect real market volatility but may negatively affect several forecasting models. Therefore, while they are kept in the dataset, they will be handled later in paragraph `3.4.3` DOK through appropriate preprocessing to minimise their impact on model performance.

##### 2.4.2.2 Energy features

In [ ]:
def boxplots(numeric_features, df):
    num_plots = len(numeric_features)
    cols = 2
    rows = math.ceil(num_plots / cols)

    plt.figure(figsize=(cols * 5, rows * 4))

    for i, feature in enumerate(numeric_features):
        plt.subplot(rows, cols, i + 1)
        sns.boxplot(y=df[feature])
        plt.title(f"Boxplot: {feature}")

    plt.tight_layout()
    plt.show()

def outliers_detection(numericFeatures, df):
    Q1 = df[numericFeatures].quantile(0.25)
    Q3 = df[numericFeatures].quantile(0.75)
    IQR = Q3 - Q1

    outliers = ((df[numericFeatures] < (Q1 - 1.5 * IQR)) | (df[numericFeatures] > (Q3 + 1.5 * IQR)))
    print("Number of outliers per numeric feature:")
    print(outliers.sum())

    outliers_cols = outliers.any()
    outliers_cols = outliers_cols[outliers_cols].index.tolist()
    print("\nColumns containing outliers:")
    print(outliers_cols)


In [ ]:
boxplots(energy_features, energy)
outliers_detection(energy_features, energy)

We performed outliers detection on all numerical features using the Interquartile Range (IQR) method and visualized it using boxplots.

Features such as `generation_fossil_gas`, `generation_hydro_pumped_storage_consumption` and `generation_other` contain a high number of outlier values.
`generation_wind_onshore`, `generation_waste`,`generation_hydro_water_reservoir` and `generation_fossil_oil` also contained a noticeable number out outliers.
`generation_biomass` contains a few outliers.
The rest of the features of `energy` dataset showed no outliers according to the IQR method.

##### 2.4.2.3 Weather features

In [ ]:
boxplots(weather_features, weather)
outliers_detection(weather_features, weather)

`pressure` and `wind_speed` contain a noticeable number of outliers.

In [ ]:
outliers_cols = ['pressure', 
                 'wind_speed', 
                 'generation_fossil_gas', 
                 'generation_hydro_pumped_storage_consumption', 
                 'generation_other', 
                 'generation_wind_onshore', 
                 'generation_waste',
                 'generation_hydro_water_reservoir',
                 'generation_biomass',
                 'generation_fossil_oil' ]

#### 2.4.3 Check data duplication

In [ ]:
energy.index.duplicated().sum()


In [ ]:
weather.index.duplicated().sum()

There are no duplicates.

---
## 3. Data Preparation

### 3.1 Data Cleaning

#### 3.1.1 Missing values

The first thing to be perfomed is to handle missing values indicated in paragraph `2.4.1`.

The missing values cannot be romoved that's why the NaNs will be filled in and columns only with NaNs will be dropped.

In [ ]:
print("Remaining NaNs in energy:", energy.isna().sum().sum())

# Drop columns that contain only NaN values
energy = energy.dropna(axis=1, how='all')

# Interpolate missing values (time-series aware)
energy.index = pd.to_datetime(energy.index, errors='coerce', utc=True)
energy = energy.interpolate(method='time')

# Forward-fill remaining NaNs (edge cases)
energy = energy.ffill().bfill()

print("Remaining NaNs in energy:", energy.isna().sum().sum())


#### 3.1.2 Handling outliers

To remove outliers identified in paragraph `2.4.2` we will perform winsorization and use **Robust Scaler** during `Data Transformation` in paragraph `3.3`.


### 3.2 Bivariate analysis

Here we will skip features that consist only of values 0.

In [ ]:
df_analysis = energy.join(weather, how="inner")

# remove columns that are entirely zero
non_zero_cols = df_analysis.columns[(df_analysis != 0).any()]

# keep only numeric features
num_df = df_analysis[non_zero_cols].select_dtypes(include="number")

# compute correlation with price
corr = num_df.corr()[target_col].drop(labels=[target_col]).sort_values()

plt.figure(figsize=(8,12))
corr.plot(kind="barh")
plt.title("Correlation with Day-Ahead Price")
plt.xlabel("Correlation")
plt.show()


The diagram shows that some variables are correlated positively, some nagetively and some are only slightly or not correlated with the target variable at all.

#### Correlation between the eletricity price and the day of the week

In [ ]:
# print(type(energy.index))
# print(energy.index[:5])

weekday_price = energy['price_day_ahead'].groupby(energy.index.to_series().dt.dayofweek).mean()
print(weekday_price)

plt.figure(figsize=(6,4))
weekday_price.plot(kind='bar')
plt.title("Average Day-Ahead Electricity Price by Day of the Week")
plt.ylabel("Price (€/MWh)")
plt.xlabel("Day of Week (0=Mon, 6=Sun)")
plt.show()


There's no strong dependency between the day of the week and the eletricity price, however the price tends to be lower on the weekends.

##### Correlation between the temperature and the eletricity price

In [ ]:
x = df_analysis["temperature"]
y = df_analysis[target_col]

# smooth curve
low = lowess(y, x, frac=0.15)

plt.figure(figsize=(10,6))
plt.scatter(x, y, s=5, alpha=0.1)
plt.plot(low[:,0], low[:,1], linewidth=3)

plt.title("LOWESS Smoothed Relationship: Temperature vs Price")
plt.xlabel("Temperature (K)")
plt.ylabel("Day-Ahead Price (€/MWh)")
plt.grid(True)
plt.show()


There's no significant correlation between the eletricity price and the temperature.

### 3.3 Data Transformation

Since all the features are numerical they don't have to be encoded in any way.

#### 3.3.1 Dropping unnecessary variables

There are variables with all 0 values. They will be dropped.

In [ ]:
df = energy.join(weather, how="inner")

dfML = df.copy()

zero_cols = dfML.columns[(dfML == 0).all()]
print("Dropped columns with all zeros:", list(zero_cols))

dfML = dfML.drop(columns=zero_cols)

dfML.info()


#### 3.3.1 Handling variables with skewed distribution

Since not all of the skewed distrubutions concern variables with positive values we will use **Yeo-Johnson** scaler.

In [ ]:
# Cast to float before transformation to avoid dtype issues
dfML = dfML.copy()

for col in skewed_cols:
    dfML[col] = dfML[col].astype(float)

In [ ]:
pt = PowerTransformer(method='yeo-johnson')

print(df[skewed_cols].dtypes)

dfML.loc[:, skewed_cols] = pt.fit_transform(dfML.loc[:, skewed_cols])

#### 3.3.2 Handling outliers

As indicated in paragraph `3.1`, we will use **Robust Scaler** to handle outliers.

In [ ]:
# Feature scaling

# Initialize the scaler
scaler = RobustScaler()

print(outliers_cols)
print(type(outliers_cols))

# Fit the scaler on the numeric features and transform
dfML.loc[:,outliers_cols] = scaler.fit_transform(dfML[outliers_cols])

### 3.4 Data Splitting

To evaluate the models used in paragraph `4` we have to split dataset into a train and a test set. We will use an 75/25 split ratio (the whole dataset represents 4 years of data and the test set will consist of last 12 months) to ensure sufficient amount of data for training while retaining enough samples for meaningful evaluation.

In [ ]:
# Last 12 months for test
split_date = dfML.index.max() - pd.DateOffset(years=1)

train_dfML = dfML[dfML.index <= split_date]
test_dfML  = dfML[dfML.index > split_date]

print("Split date:", split_date)
print("Train:", train_dfML.shape)
print("Test:", test_dfML.shape)

### 3.5 Feature selection

#### Embedded method - Lasso regularization

In [ ]:
def lasso_regularization(df, target_col="price_day_ahead"):

    X = df.drop(columns=[target_col]).copy()
    y = df[target_col].copy()

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    lasso = Lasso(alpha=0.01, random_state=42, max_iter=5000)

    sel_ = SelectFromModel(lasso)
    sel_.fit(X_scaled, y)

    selected_feat = X.columns[sel_.get_support()]
    removed_feat = X.columns[~sel_.get_support()]

    print(f"Selected features ({len(selected_feat)}):")
    print(selected_feat.tolist())

    print(f"\nRemoved features ({len(removed_feat)}):")
    print(removed_feat.tolist())

    return selected_feat

In [ ]:
Lasso_SelectedColumns = lasso_regularization(train_dfML)

Lasso_SelectedColumns

In [ ]:
train_dfML = train_dfML[list(Lasso_SelectedColumns) + [target_col]]
test_dfML  = test_dfML[list(Lasso_SelectedColumns) + [target_col]]

TODO

With these methods, we prepared a dataframes for future testing and tuning to compare different models and select the one with the best performance.

---
## 4. Modelling

### 4.1 Statistical Models

TODO

### 4.2 Machine Learning Models

comment

#### 4.2.1 Build supervised sliding-window datasets

We will be using two functions two build sliding-window datasets. `make_sliding_window` is used for single-step predicitons and `slideWindow` for multi-step predictions. Later on these functions will be used to build the sliding-window datasets after choosing the best window size in paragraph `4.2.2`.

In [ ]:
def make_sliding_window(series, window_size, horizon=1): # for single-step
    X, y = [], []
    for i in range(len(series) - window_size - horizon + 1):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size + horizon - 1])
    return np.array(X), np.array(y)


def slideWindow(time_series, window_in, horizon): # for multi-step
    if (len(time_series) - window_in - horizon + 1) <= 0:
       raise ValueError("Time series too short for window size or horizon.")
    
    d = time_series.values
    X, y = [], []

    for start in range(len(time_series)-window_in):
        end = start + window_in
        out = end + horizon
        X.append(d[start:end].reshape(-1))
        y.append(d[end:out].ravel()) # ravel() is equivalent to reshape(-1): returnscontiguous flattened array
        cols_x = [f'x{i}' for i in range(1, window_in+1)]
        cols_y = [f'y{i}' for i in range(1, horizon+1)]
        df_X = pd.DataFrame(X, columns=cols_x)
        df_y = pd.DataFrame(y, columns=cols_y)

    return pd.concat([df_X, df_y], axis=1).dropna()

def slideWindow2(time_series, window_in, horizon):
    if (len(time_series) - window_in - horizon + 1) <= 0:
        raise ValueError("Time series too short for window size or horizon.")

    d = time_series.values
    n_samples = len(d) - window_in - horizon + 1

    X = np.empty((n_samples, window_in))
    y = np.empty((n_samples, horizon))

    for i in range(n_samples):
        X[i] = d[i : i + window_in]
        y[i] = d[i + window_in : i + window_in + horizon]

    cols_x = [f"x{i}" for i in range(1, window_in + 1)]
    cols_y = [f"y{i}" for i in range(1, horizon + 1)]

    return pd.DataFrame(
        np.hstack([X, y]),
        columns=cols_x + cols_y
    )



# HERE

#### 4.2.2 Several window sizes comparison

Now we will compare three window sizes (one day, two days and one week) to find the most efficient one. For that we will use Random Forest because it's the least sensitive to data changes and variance is reduced by averaging.
To compare the results we will use MAE - Mean Aboslute Error - because it has low sensitivity.

In [ ]:
window_sizes = [24, 48, 168]   # 1 day, 2 days, 1 week
horizons = [1, 24] # the number of time steps into the future to forecast at any point in time

tscv = TimeSeriesSplit(n_splits=5)

results = []

for w in window_sizes:

    X, y = make_sliding_window(train_dfML[target_col].values, window_size=w, horizon=1)

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = RandomForestRegressor(
            n_estimators=50,
            max_depth=10,
            random_state=42,
            n_jobs=-1 # enable paralelism
        )

        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)

        results.append({
            "window": w,
            "fold": fold,
            "MAE": mae
        })
        print(f"Window {w}, Fold {fold}")

results_df = pd.DataFrame(results)
print(results_df.groupby("window")["MAE"].mean())


Mean Aboslute Error (MAE) is the lowest for 24-hour long window.

In [ ]:
BEST_WINDOW = 24

In [ ]:
X_train, y_train = make_sliding_window(
    train_dfML[target_col].values,
    window_size=BEST_WINDOW,
    horizon=1
)

print("One Day Sliding Window Created for 1 step") 

In [ ]:
df_sw_one_day_multi_step = slideWindow2(
        train_dfML[target_col],
        window_in=BEST_WINDOW,
        horizon=24
    )

print("One Day Sliding Window Created for 24 step") # TO BE CREATED LATER

#### 4.2.3 Model selection and tuning

Model selection will be performed for 1-step predicitons.

In [ ]:
models_stage1 = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),

    "DecisionTree": DecisionTreeRegressor(random_state=42),

    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("model", KNeighborsRegressor())
    ]),

    "SVR": Pipeline([
        ("scaler", StandardScaler()),
        ("model", SVR())
    ]),

    "Bagging": BaggingRegressor(random_state=42),

    "GradientBoosting": GradientBoostingRegressor(random_state=42),

    "RandomForest": RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    ),

    "XGBoost": XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    ),

    "LightGBM": LGBMRegressor(
        random_state=42
    )
}

In [ ]:
def safe_mape(y_true, y_pred, eps=1e-8):
    y_true = np.maximum(np.abs(y_true), eps)
    return np.mean(np.abs((y_true - y_pred) / y_true))


In [ ]:
scoring = {
    "mae": "neg_mean_absolute_error",
    "rmse": make_scorer(
        lambda y, yhat: np.sqrt(mean_squared_error(y, yhat)),
        greater_is_better=False
    ),
    "mape": make_scorer(
        safe_mape,
        greater_is_better=False
    )
}

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

stage1_results = []

for name, model in models_stage1.items():
    fold_mae = []
    fold_rmse = []
    fold_mape = []

    print(f"\nTraining model: {name}")

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
        print(f"  Fold {fold}/5")

        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)

        fold_mae.append(mean_absolute_error(y_val, y_pred))
        fold_rmse.append(np.sqrt(mean_squared_error(y_val, y_pred)))
        fold_mape.append(
            np.mean(np.abs((y_val - y_pred) / np.maximum(np.abs(y_val), 1e-8)))
        )

    stage1_results.append({
        "Model": name,
        "MAE": np.mean(fold_mae),
        "RMSE": np.mean(fold_rmse),
        "MAPE": np.mean(fold_mape),
    })

    print(f"  → MAE:  {np.mean(fold_mae):.3f}")
    print(f"  → RMSE: {np.mean(fold_rmse):.3f}")
    print(f"  → MAPE: {np.mean(fold_mape):.3f}")


In [ ]:
results_stage1 = (
    pd.DataFrame(stage1_results)
      .sort_values(by=["MAE", "RMSE", "MAPE"])
      .reset_index(drop=True)
)

print("\nFinal results:")
print(results_stage1)


Before tuning:

| Model            | MAE    | RMSE   | MAPE   |
| ---------------- | ------ | ------ | ------ |
| LinearRegression | 2.0272 | 3.1191 | 0.0561 |
| RandomForest     | 2.2178 | 3.4819 | 0.0657 |
| LightGBM         | 2.2992 | 3.5899 | 0.0693 |
| GradientBoosting | 2.3319 | 3.6322 | 0.0688 |
| Bagging          | 2.3822 | 3.6588 | 0.0676 |
| XGBoost          | 2.4362 | 3.7724 | 0.0773 |
| SVR              | 3.1489 | 5.6502 | 0.1291 |
| DecisionTree     | 3.4551 | 5.2995 | 0.1020 |
| KNN              | 3.9266 | 5.5235 | 0.1241 |


#### 4.2.4 Hyperparameter tuning

Hyperparameter tuning will be performed for 1-step predictions.

svr musi byc sekwencyjnie

In [ ]:
param_grids = {
    "DecisionTree": {
        "max_depth": [3, 5, 10, None],
        "min_samples_leaf": [1, 5, 10]
    },

    "KNN": {
        "model__n_neighbors": [3, 5, 10, 20]
    },

    "SVR": {
        "model__C": [1, 10],
        "model__gamma": ["scale"]
    },

    "Bagging": {
        "n_estimators": [50, 100]
    },

    "GradientBoosting": {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5]
    },

    "RandomForest": {
        "n_estimators": [100, 200],
        "max_depth": [5, 10, None]
    },

    "XGBoost": {
        "n_estimators": [200, 400],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5]
    },

    "LightGBM": {
        "n_estimators": [200, 400],
        "learning_rate": [0.05, 0.1],
        "num_leaves": [31, 63]
    }
}


In [ ]:
tuning_results = []
best_estimators = {}

tscv = TimeSeriesSplit(n_splits=5)

for name, model in models_stage1.items():
    print(f"\nTuning model: {name}")

    if name == "LinearRegression":
        print("No tuning perfomed")
        continue

    if name in ["SVR", "LightGBM"]:
        grid = GridSearchCV(
            estimator=model,
            param_grid=param_grids.get(name, {}),
            cv=tscv,
            scoring=scoring,     
            refit="mae",         
            n_jobs=1
        )
    elif name in ["RandomForest", "XGBoost"]:
        grid = GridSearchCV(
            estimator=model,
            param_grid=param_grids.get(name, {}),
            cv=tscv,
            scoring=scoring,    
            refit="mae"
        )
    else:
        grid = GridSearchCV(
            estimator=model,
            param_grid=param_grids.get(name, {}),
            cv=tscv,
            scoring=scoring,     
            refit="mae",
            n_jobs=-1
        )

    grid.fit(X_train, y_train)

    best_idx = grid.best_index_

    best_mae = -grid.cv_results_["mean_test_mae"][best_idx]
    best_rmse = -grid.cv_results_["mean_test_rmse"][best_idx]
    best_mape = -grid.cv_results_["mean_test_mape"][best_idx]

    tuning_results.append({
        "Model": name,
        "Best_MAE": best_mae,
        "Best_RMSE": best_rmse,
        "Best_MAPE": best_mape,
        "Best_Params": grid.best_params_
    })

    best_estimators[name] = grid.best_estimator_

    print(f"  → MAE:  {best_mae:.3f}")
    print(f"  → RMSE: {best_rmse:.3f}")
    print(f"  → MAPE: {best_mape:.3f}")


In [ ]:
results_tuning = (
    pd.DataFrame(tuning_results)
      .sort_values("Best_MAE")
      .reset_index(drop=True)
)

print("\nFinal results:")
print(results_tuning)


After tuning:

| Model            | MAE       | RMSE      | MAPE       | Best Parameters                                       |
| ---------------- | --------- | --------- | ---------- | ----------------------------------------------------- |
| **RandomForest** | **2.211** | **3.470** | **0.0658** | `max_depth=None, n_estimators=200`                    |
| Bagging          | 2.221     | 3.479     | 0.0654     | `n_estimators=100`                                    |
| LightGBM         | 2.263     | 3.544     | 0.0684     | `learning_rate=0.05, n_estimators=200, num_leaves=31` |
| GradientBoosting | 2.274     | 3.621     | 0.0703     | `learning_rate=0.05, max_depth=5, n_estimators=200`   |
| XGBoost          | 2.279     | 3.595     | 0.0707     | `learning_rate=0.05, max_depth=5, n_estimators=200`   |
| SVR              | 2.527     | 4.664     | 0.0832     | `C=10, gamma=scale`                                   |
| DecisionTree     | 2.717     | 4.049     | 0.0793     | `max_depth=10, min_samples_leaf=10`                   |
| KNN              | 3.875     | 5.434     | 0.1281     | `n_neighbors=10`                                      |


Among the evaluated machine-learning models, Random Forest achieved the best overall performance, obtaining the lowest MAE, RMSE, and MAPE. Ensemble methods generally outperformed single learners, while distance-based (KNN) and kernel-based (SVR) models performed worse. 

***Comparison with the baseline: Seasonal Naïve***

In [ ]:
def naive_last_value_cv(X_train, y_train, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    maes, rmses, mapes = [], [], []

    for _, val_idx in tscv.split(X_train):
        X_val = X_train[val_idx]
        y_val = y_train[val_idx]

        # Naive forecast = last value in the window
        y_pred = X_val[:, -1]

        maes.append(mean_absolute_error(y_val, y_pred))
        rmses.append(np.sqrt(mean_squared_error(y_val, y_pred)))
        mapes.append(
            np.mean(np.abs((y_val - y_pred) / np.maximum(np.abs(y_val), 1e-8)))
        )

    return {
        "Model": "Naive (Last value)",
        "MAE": np.mean(maes),
        "RMSE": np.mean(rmses),
        "MAPE": np.mean(mapes),
    }


In [ ]:
baseline_results = naive_last_value_cv(X_train, y_train)
baseline_results

In [ ]:
SEASONAL_LAG = 24   # change if your data is different

def seasonal_naive_cv(X, y, seasonal_lag=24, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    maes, rmses, mapes = [], [], []

    for train_idx, val_idx in tscv.split(X):
        X_val = X[val_idx]
        y_val = y[val_idx]

        # Seasonal naive forecast
        # Take value from t - seasonal_lag inside the window
        y_pred = X_val[:, -seasonal_lag]

        maes.append(mean_absolute_error(y_val, y_pred))
        rmses.append(np.sqrt(mean_squared_error(y_val, y_pred)))
        mapes.append(
            np.mean(
                np.abs((y_val - y_pred) / np.maximum(np.abs(y_val), 1e-8))
            )
        )

    return {
        "Model": "SeasonalNaive",
        "MAE": np.mean(maes),
        "RMSE": np.mean(rmses),
        "MAPE": np.mean(mapes),
    }



In [ ]:
baseline_result = seasonal_naive_cv(
    X_train,
    y_train,
    seasonal_lag=24,
    n_splits=5
)

baseline_df = pd.DataFrame([baseline_result])


In [ ]:
# final results

results_before = (
    pd.DataFrame(stage1_results)
    .sort_values(by=["MAE", "RMSE", "MAPE"])
    .rename(columns={
        "MAE": "MAE_before",
        "RMSE": "RMSE_before",
        "MAPE": "MAPE_before"
    })
    .reset_index(drop=True)
)

results_after = (
    pd.DataFrame(tuning_results)
    .rename(columns={
        "Best_MAE": "MAE_after",
        "Best_RMSE": "RMSE_after",
        "Best_MAPE": "MAPE_after"
    })
    .drop(columns="Best_Params")
)

comparison_table = (
    results_before
    .merge(results_after, on="Model", how="left")
    .sort_values("MAE_after")
    .reset_index(drop=True)
)

final_table = pd.concat(
    [baseline_df, comparison_table],
    ignore_index=True
)

final_table = final_table.sort_values(
    by=["MAE_after", "RMSE_after", "MAPE_after"],
    na_position="last"
).reset_index(drop=True)

print("\nFinal results (including baseline):")
print(final_table)


#### 4.2.5 Perform predictions

##### 4.2.5.1 Single-step

##### 4.2.5.2 Multi-step (24-step)

#### 4.2.5.3 Plots comparing predictions vs. real test data

### 4.3 Deep Learning Models

TODO

LSTM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# ==========================================
# 1. Data Preparation
# ==========================================
WINDOW_SIZE = 48  # Lookback period (e.g., 48 hours)
BATCH_SIZE = 32   # Hyperparameter to tune
EPOCHS = 50       # Max epochs

scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_dfML)
test_scaled = scaler.transform(test_dfML)

X_train_full, y_train_full = make_sliding_window(train_scaled, WINDOW_SIZE, horizon=1)
X_test, y_test = make_sliding_window(test_scaled, WINDOW_SIZE, horizon=1)

y_train_full = y_train_full[:, 0]
y_test = y_test[:, 0]

val_split = int(len(X_train_full) * 0.8)
X_train, y_train = X_train_full[:val_split], y_train_full[:val_split]
X_val, y_val = X_train_full[val_split:], y_train_full[val_split:]

print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}")

In [ ]:
# ==========================================
# 2. LSTM Model Definition
# ==========================================
def build_lstm_model(input_shape):
    lstm_model = Sequential()

    # Input Layer
    lstm_model.add(Input(shape=input_shape))

    # LSTM Layer 1
    lstm_model.add(LSTM(units=50, return_sequences=True))
    lstm_model.add(Dropout(0.2))

    # LSTM Layer 2
    lstm_model.add(LSTM(units=25, return_sequences=False))
    lstm_model.add(Dropout(0.2))

    # Output Layer
    lstm_model.add(Dense(1))

    optimizer = Adam(learning_rate=0.0001)
    lstm_model.compile(optimizer=optimizer, loss='mse')
    return lstm_model


lstm_model = build_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

In [ ]:
# ==========================================
# 3. Training
# ==========================================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

history = lstm_model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)


plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

In [ ]:
# ==========================================
# 4. Evaluation & Metrics
# ==========================================
y_pred_scaled = lstm_model.predict(X_test)

def inverse_transform_target(pred_array, scaler, n_features):
    dummy = np.zeros((len(pred_array), n_features))
    dummy[:, 0] = pred_array.flatten()
    return scaler.inverse_transform(dummy)[:, 0]

y_pred_real = inverse_transform_target(y_pred_scaled, scaler, train_dfML.shape[1])
y_test_real = inverse_transform_target(y_test.reshape(-1, 1), scaler, train_dfML.shape[1])

mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
non_zero_mask = y_test_real != 0
mape = np.mean(np.abs((y_test_real[non_zero_mask] - y_pred_real[non_zero_mask]) / y_test_real[non_zero_mask])) * 100

print(f"\nModel Performance:")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.4f}%")

plt.figure(figsize=(15, 6))
plt.plot(y_test_real[:200], label='Actual Price', color='blue')
plt.plot(y_pred_real[:200], label='Predicted Price', color='red', linestyle='--')
plt.title('LSTM Forecast: Actual vs Predicted (First 200 Hours)')
plt.legend()
plt.show()

RESULTS ANALYSIS

The Long Short-Term Memory (LSTM) network demonstrated strong performance in capturing the temporal dependencies of electricity prices, achieving an MAE of 0.0959 and an RMSE of 0.1532. The high MAPE (82.15%) is attributed to the presence of near-zero prices in the test set rather than poor predictive quality.

Visually, the LSTM predictions track the actual test data very closely, successfully replicating the daily seasonality and general trends of the market. A slight "reaction delay" was observed, which is expected in autoregressive models that rely on immediate past windows. While the model handles standard volatility well, it occasionally struggles with extreme, rapid fluctuations (drastic down-and-up movements), where it tends to smooth the curve rather than capturing the full amplitude of the sharpest spikes.

The training process was stable, with training and validation loss curves converging naturally. The EarlyStopping mechanism was not triggered, and the optimal model weights were recorded at epoch 49 (out of 50). This continuous improvement throughout the training cycle indicates that the model had appropriate capacity and learning rate settings for this dataset, avoiding both premature convergence and overfitting.

GRU

In [ ]:
from tensorflow.keras.layers import GRU
from tensorflow.keras.optimizers import Adam

In [ ]:
# ==========================================
# 1. GRU Model Definition
# ==========================================
def build_gru_model(input_shape):
    gru_model = Sequential()

    # Input Layer
    gru_model.add(Input(shape=input_shape))

    # GRU Layer 1
    gru_model.add(GRU(units=50, return_sequences=True))
    gru_model.add(Dropout(0.2))

    # GRU Layer 2
    gru_model.add(GRU(units=25, return_sequences=False))
    gru_model.add(Dropout(0.2))

    # Output Layer
    gru_model.add(Dense(1))

    optimizer = Adam(learning_rate=0.0001)
    gru_model.compile(optimizer=optimizer, loss='mse')
    return gru_model


gru_model = build_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]))

In [ ]:
# ==========================================
# 2. Training
# ==========================================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

history_gru = gru_model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

plt.figure(figsize=(10, 6))
plt.plot(history_gru.history['loss'], label='Training Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title('GRU Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

In [ ]:
# ==========================================
# 3. Evaluation
# ==========================================
y_pred_scaled_gru = gru_model.predict(X_test)

y_pred_real_gru = inverse_transform_target(y_pred_scaled_gru, scaler, train_dfML.shape[1])

mae_gru = mean_absolute_error(y_test_real, y_pred_real_gru)
rmse_gru = np.sqrt(mean_squared_error(y_test_real, y_pred_real_gru))
non_zero_mask = y_test_real != 0
mape_gru = np.mean(np.abs((y_test_real[non_zero_mask] - y_pred_real_gru[non_zero_mask]) / y_test_real[non_zero_mask])) * 100

print(f"\nGRU Model Performance:")
print(f"MAE:  {mae_gru:.4f}")
print(f"RMSE: {rmse_gru:.4f}")
print(f"MAPE: {mape_gru:.4f}%")

plt.figure(figsize=(15, 6))
plt.plot(y_test_real[:200], label='Actual Price', color='black', alpha=0.7)
plt.plot(y_pred_real[:200], label='LSTM Prediction', color='red', linestyle='--', alpha=0.7)
plt.plot(y_pred_real_gru[:200], label='GRU Prediction', color='green', linestyle='-.', alpha=0.7)
plt.title('Comparison: LSTM vs GRU vs Actual (First 200 Hours)')
plt.legend()
plt.show()

GRU Analysis

The Gated Recurrent Unit (GRU) model outperformed the LSTM on this dataset, achieving a lower MAE of 0.0822 (vs. 0.0959 for LSTM) and RMSE of 0.1381 (vs. 0.1532 for LSTM). This indicates that the GRU provided a slightly more accurate fit to the test data overall. As expected, the training time was shorter, 14 minutes insted of 18 for LSTM.

Visually, the GRU predictions followed a very similar trajectory to the LSTM, closely tracking the actual price curve. This suggests that both recurrent architectures successfully captured the underlying temporal patterns of the electricity market. The fact that the simpler GRU architecture yielded better metrics suggests that the additional complexity of the LSTM (extra gates and separate cell state) was not necessary for this specific 1-step forecast task and may have introduced slight inefficiencies or noise.

Similar to the LSTM, the GRU showed stable learning and continued to improve throughout the training process, with the best model weights saved at the final epoch 50. This confirms that the model did not overfit and could potentially benefit from an even longer training duration, though the current performance is already superior to the LSTM baseline.

24-Step Ahead Model

In [ ]:
# ==========================================
# 1. Data Preparation for 24-Step Ahead
# ==========================================
def create_24h_window(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size - 24 + 1):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size: i + window_size + 24, 0])
    return np.array(X), np.array(y)

WINDOW_SIZE = 48

X_train_24, y_train_24 = create_24h_window(train_scaled, WINDOW_SIZE)
X_test_24, y_test_24 = create_24h_window(test_scaled, WINDOW_SIZE)

val_split_24 = int(len(X_train_24) * 0.8)
X_train_sub_24, y_train_sub_24 = X_train_24[:val_split_24], y_train_24[:val_split_24]
X_val_24, y_val_24 = X_train_24[val_split_24:], y_train_24[val_split_24:]

print(f"Input Shape: {X_train_sub_24.shape}")
print(f"Target Shape: {y_train_sub_24.shape}")

In [ ]:
# ==========================================
# 2. LSTM Model for 24-Step Output
# ==========================================
model_24 = Sequential()
# Input Layer
model_24.add(Input(shape=(X_train_sub_24.shape[1], X_train_sub_24.shape[2])))

# LSTM24 Layer 1
model_24.add(LSTM(128, return_sequences=True))
model_24.add(Dropout(0.2))

# LSTM24 Layer 2
model_24.add(LSTM(68, return_sequences=False))
model_24.add(Dropout(0.2))

# OUTPUT LAYER
model_24.add(Dense(24))

model_24.compile(optimizer='adam', loss='mse')

In [ ]:
# ==========================================
# 3. Training
# ==========================================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

history_24 = model_24.fit(
    X_train_sub_24, y_train_sub_24,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_24, y_val_24),
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# ==========================================
# 4. Evaluation
# ==========================================
y_pred_scaled_24 = model_24.predict(X_test_24)

n_samples = y_pred_scaled_24.shape[0]
n_steps = 24
n_features = train_dfML.shape[1]

flat_preds = y_pred_scaled_24.reshape(-1, 1)
dummy_preds = np.zeros((len(flat_preds), n_features))
dummy_preds[:, 0] = flat_preds.flatten()
flat_real_preds = scaler.inverse_transform(dummy_preds)[:, 0]

y_pred_real_24 = flat_real_preds.reshape(n_samples, n_steps)

flat_actuals = y_test_24.reshape(-1, 1)
dummy_actuals = np.zeros((len(flat_actuals), n_features))
dummy_actuals[:, 0] = flat_actuals.flatten()
flat_real_actuals = scaler.inverse_transform(dummy_actuals)[:, 0]
y_test_real_24 = flat_real_actuals.reshape(n_samples, n_steps)

mae_24 = mean_absolute_error(y_test_real_24, y_pred_real_24)
rmse_24 = np.sqrt(mean_squared_error(y_test_real_24, y_pred_real_24))
mask_24 = y_test_real_24 != 0
mape_24 = np.mean(np.abs((y_test_real_24[mask_24] - y_pred_real_24[mask_24]) / y_test_real_24[mask_24])) * 100

print(f"\n24-Step Ahead Model Performance:")
print(f"MAE:  {mae_24:.4f}")
print(f"RMSE: {rmse_24:.4f}")
print(f"MAPE: {mape_24:.4f}%")


sample_idx = 0
plt.figure(figsize=(10, 5))
plt.plot(range(24), y_test_real_24[sample_idx], marker='o', label='Actual Day')
plt.plot(range(24), y_pred_real_24[sample_idx], marker='x', linestyle='--', label='Predicted Day')
plt.title(f'24-Hour Forecast for Sample #{sample_idx}')
plt.xlabel('Hour of Day')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# ==========================================
# 5. Continuous Plot (Day-Ahead Simulation)
# ==========================================

stitched_pred = y_pred_real_24[::24].flatten()
stitched_actual = y_test_real_24[::24].flatten()

plot_hours = 240 # 10 days

plot_hours = min(plot_hours, len(stitched_pred))

plt.figure(figsize=(15, 6))

plt.plot(stitched_actual[:plot_hours], label='Actual Price', color='blue', alpha=0.6)

plt.plot(stitched_pred[:plot_hours], label='Day-Ahead Prediction (Stitched)',
         color='orange', linestyle='--', marker='.', markersize=5)

plt.title(f'Day-Ahead Forecast: Continuous View (First {plot_hours} Hours)')
plt.xlabel('Hours')
plt.ylabel('Electricity Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

mae_stitched = mean_absolute_error(stitched_actual, stitched_pred)
print(f"Realistic Day-Ahead Operation MAE: {mae_stitched:.4f}")

24-Step LSTM Analysis

Predicting a full 24-hour horizon proved to be significantly more challenging than the 1-step task. The model achieved an MAE of 0.2650 and RMSE of 0.3563. These errors are roughly 3x higher than the 1-step GRU/LSTM models (MAE ~0.08), reflecting the inherent difficulty of forecasting an entire day without the ability to correct course using intermediate real values.

On a larger scale, the model demonstrates a basic ability to follow the general market trend, confirming it learned the overall weekly and daily seasonality. A major limitation observed is a substantial "phase shift" or lag. When a rapid price drop occurs, the model often predicts this drop only after the event has passed (likely with a 24-hour delay). This suggests the model is relying heavily on persistence (repeating yesterday's pattern) rather than anticipating changes based on exogenous features like weather. Visually, the predictions are frequently "off" from the actual values, lacking the precision seen in the 1-step models. This is consistent with the EarlyStopping occurring at epoch 15 (best weights at epoch 5), indicating the model struggled to find a complex pattern and converged quickly to a simpler, safer suboptimal solution.

Hyperparameter tuning

In [ ]:
batch_sizes = [16, 32, 64]
results = {}

for bs in batch_sizes:
    print(f"Testing Batch Size: {bs}")
    model = build_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train
    history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=bs,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=0
    )

    best_val_loss = min(history.history['val_loss'])
    results[bs] = best_val_loss
    print(f"Batch Size {bs} - Best Val Loss: {best_val_loss:.5f}")

print("Best Configuration:", min(results, key=results.get))

In [ ]:
def build_tunable_lstm(input_shape, units, num_layers, dropout_rate=0.2):
    tunable_lstm_model = Sequential()

    # Input Layer
    tunable_lstm_model.add(Input(shape=input_shape))

    # First Layer (Always present)
    return_seq = (num_layers > 1)
    tunable_lstm_model.add(LSTM(units=units, return_sequences=return_seq))
    tunable_lstm_model.add(Dropout(dropout_rate))

    # Second Layer (Conditional)
    if num_layers == 2:
        tunable_lstm_model.add(LSTM(units=units // 2, return_sequences=False))
        tunable_lstm_model.add(Dropout(dropout_rate))

    # Output Layer
    tunable_lstm_model.add(Dense(1))

    optimizer = Adam(learning_rate=0.0001)
    tunable_lstm_model.compile(optimizer=optimizer, loss='mse')
    return tunable_lstm_model

In [ ]:
# ==========================================
# Hyperparameter Tuning: Units & Layers
# ==========================================

units_options = [32, 64, 128]
layers_options = [1, 2]

tuning_results = []

print("Starting Hyperparameter Tuning...")
print("-" * 50)

for units in units_options:
    for layers in layers_options:
        print(f"Testing: {units} Units | {layers} Layers...")

        model = build_tunable_lstm(
            input_shape=(X_train.shape[1], X_train.shape[2]),
            units=units,
            num_layers=layers
        )

        history = model.fit(
            X_train, y_train,
            epochs=30,
            batch_size=32, # Using your fixed best batch size
            validation_data=(X_val, y_val),
            callbacks=[early_stop],
            verbose=0
        )

        best_val_loss = min(history.history['val_loss'])

        tuning_results.append({
            'units': units,
            'layers': layers,
            'val_loss': best_val_loss
        })

        print(f"   -> Result: Best Val Loss = {best_val_loss:.5f}")

print("-" * 50)
print("Tuning Complete.")

In [ ]:
results_df = pd.DataFrame(tuning_results)

results_df = results_df.sort_values(by='val_loss', ascending=True)

print("\nTop 3 Configurations:")
print(results_df.head(3))

best_config = results_df.iloc[0]
print(f"\nWINNER: LSTM with {int(best_config['units'])} Units and {int(best_config['layers'])} Layers.")

Hyperparameter Analysis

The tuning process revealed a clear preference for a simpler architecture. The configuration with 32 units achieved the lowest validation loss (0.000856), outperforming larger configurations like 128 units (0.000904) or 64 units (0.000981). The differences are extremly slow and multiple test showed that the results may vary depending on test.

The fact that 1-layer models consistently outperformed 2-layer models (which did not appear in the top 3) suggests that the features in this dataset (price, weather, generation) are strong enough that a deeper, more complex network is not required. Adding a second layer likely introduced unnecessary noise or optimization difficulties rather than learning new abstractions.

Selecting the 32-unit model not only improves accuracy but also significantly reduces the computational cost, making the model faster to train and lighter to deploy.

In [ ]:
# ==========================================
# Seasonal Naive
# ==========================================
seasonality = 168
y_naive_pred = []
y_naive_real = []

test_values = test_dfML.iloc[:, 0].values # Assuming price is col 0

for i in range(seasonality, len(test_values)):
    y_naive_pred.append(test_values[i - seasonality])
    y_naive_real.append(test_values[i])

mae_naive = mean_absolute_error(y_naive_real, y_naive_pred)
rmse_naive = np.sqrt(mean_squared_error(y_naive_real, y_naive_pred))
print(f"Baseline (Seasonal Naive) - MAE: {mae_naive:.4f}, RMSE: {rmse_naive:.4f}")

plt.figure(figsize=(15, 6))
plt.plot(y_naive_real[:240], label='Actual Price', color='tab:blue', alpha=0.7)
plt.plot(y_naive_pred[:240], label='10-Day Recursive Forecast', color='tab:orange', linestyle='--')

plt.title(f'10-Day Recursive Forecast (LSTM)')
plt.ylabel('Price (€)')
plt.xlabel('Hours (0 to 240)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

Naive Analysis

The Seasonal Naive baseline performed significantly worse than the Deep Learning models, yielding an MAE of 0.4179 and RMSE of 0.5468. In comparison, the best-performing model (GRU) achieved an MAE of roughly 0.08, representing an improvement of approximately 80% over this baseline. This huge gap confirms that the project's complexity was justified.

The graph appears to "predict something" in certain areas because electricity demand does have a strong weekly structure. The areas where the prediction looks "random" or fails completely represent times when exogenous factors (weather changes, wind generation, holidays) shifted the price away from the historical average. Because the Naïve model blindly copies the previous week's value , it cannot account for these dynamic changes, leading to large errors whenever the weather pattern changes from one week to the next.

The poor performance of the Seasonal Naïve model demonstrates that electricity prices are not merely repetitive cycles. They are driven by complex, non-linear dependencies on weather and generation data, which only the Deep Learning models (LSTM/GRU) were able to successfully learn and predict.

---
## 6. Conclusions and Recommendations

### 6.1 Summary of Findings


### 6.2 Best Model Selection

### 6.3 Future Work


### 6.4 Lessons Learned


---

## Project Notes and Team Collaboration

### Team Members

- Julia Kardasz 1250264
- Mateusz Nowak 1250296
- Emilia Pawlowska 1250230